# **Implementing Histogram Task Solution by Wael KABOUK, ID:Y235050005**

##Preparing CUDA Enviorement

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-osx1sgdq
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-osx1sgdq
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Implementing The Code And Writing it To A File



In [38]:
%%writefile computeHistogram.cu
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <chrono>
#include <time.h>

#define CHECK(call)                                            \
{                                                               \
    const cudaError_t error = call;                             \
    if (error != cudaSuccess)                                   \
    {                                                           \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__); \
        fprintf(stderr, "code: %d, reason: %s\n", error,        \
        cudaGetErrorString(error));                     \
        exit(EXIT_FAILURE);                                             \
    }                                                           \
}

// Kernel function for implementing histogram using shared memory
__global__ void gpuHistogram(const int *inputData, int *outputHistogram)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    __shared__ int sharedHistogram[256];
    sharedHistogram[threadIdx.x] = 0;
    __syncthreads();


  atomicAdd(&(sharedHistogram[inputData[i]]), 1);
  __syncthreads();

  atomicAdd(&(outputHistogram[threadIdx.x]), sharedHistogram[threadIdx.x]);
}

// Function to generate a random integer in a given range
int randInt(int from, int to)
{
    return rand() % (to - from + 1) + from;
}

// Function to print the first cpu/gpu 10 computations
void printGpuCpuHistograms(int* histogram, int *testHistogram)
{
    printf("Bins\tGPU\tCPU\n");
    for (int i = 0; i < 10; i++) {
        printf("Bin[%d]: %d \t %d", (i + 1), histogram[i], testHistogram[i]);
        printf("\n");

    }
    printf("\n");
}

// Function for histogram computation on the CPU
void cpuHistogram(const int* inputData, int* outputHistogram, int elementsNum)
{
    for (int i = 0; i < elementsNum; i++) {
        outputHistogram[inputData[i]]++;
    }
}

int main(int argc, char** argv)
{
    // Initialize random seed
    srand((unsigned)time(NULL));

    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Vector length and size calculation
    int numElements = pow(atoi(argv[1]), atoi(argv[2]));
    size_t size = numElements * sizeof(int);
    size_t bin_size = 256 * sizeof(int);

    printf("Histogram Computation of %d elements\n", numElements);

    // Allocate host input data and output histogram vectors
    int* h_inputData = (int*)malloc(size);

    int* h_histogram = (int*)malloc(bin_size);

    int* h_testHistogram = (int*)malloc(bin_size);


    // Verify host vector allocations
    if (h_inputData == NULL || h_histogram == NULL)
    {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize host input data vector
    for (int i = 0; i < numElements; i++)
    {
        h_inputData[i] = randInt(1, 100);

    }

    // Allocate device input and output vectors
    int* d_inputData = NULL;
    int* d_histogram = NULL;


    err = cudaMalloc((void**)&d_inputData, size);
    CHECK(err);

    err = cudaMalloc((void**)&d_histogram, bin_size);
    CHECK(err);

    // Copy host input data vector to device
    printf("Copy input data from host memory to CUDA device\n");
    err = cudaMemcpy(d_inputData, h_inputData, size, cudaMemcpyHostToDevice);
    CHECK(err);

    // CUDA event setup for timing
    cudaEvent_t start, stop;
    float elapsedTime;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);

    // Launch the histogram computation CUDA Kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (numElements + threadsPerBlock - 1) / threadsPerBlock;

    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    gpuHistogram << < blocksPerGrid, threadsPerBlock >> > (d_inputData, d_histogram);
    err = cudaGetLastError();
    CHECK(err);

    // Record CUDA event stop time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    // Calculate and print GPU elapsed time
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("Time on device: %f ms\n", elapsedTime);



    clock_t cpuStart, cpuStop;
    float cpuDuration;

    // Record the start time for CPU
    cpuStart = clock();

    // Perform histogram computation on CPU
    cpuHistogram(h_inputData, h_testHistogram, numElements);

    // Record the stop time for CPU
    cpuStop = clock();

    // Calculate elapsed time on the CPU
    cpuDuration = ((float)(cpuStop - cpuStart)) * 1000.0 / CLOCKS_PER_SEC;
    printf("Time on CPU: %f ms\n", cpuDuration);

    // Calculate and print speedup
    float speedup = cpuDuration / elapsedTime;
    printf("Speed-up: %.2f\n", speedup);

    // Copy device result histogram vector to host
    printf("Copy output data from CUDA device to host memory\n");
    err = cudaMemcpy(h_histogram, d_histogram, bin_size, cudaMemcpyDeviceToHost);
    CHECK(err);

    //Printing first 10 histogram results
    printf("GPU / CPU Histogram First 10 Computations Comparison\n");
    printGpuCpuHistograms(h_histogram, h_testHistogram);





    // Free device global memory
    err = cudaFree(d_inputData);
    CHECK(err);

    err = cudaFree(d_histogram);
    CHECK(err);


    // Free host memory
    free(h_inputData);
    free(h_histogram);


    printf("Done\n");
    return 0;
}


Overwriting computeHistogram.cu


## Compiling The code

In [39]:
!nvcc -o computeHistogram computeHistogram.cu

## Executing the code with an input value of N = 2^10

In [40]:
!./computeHistogram 2 10

Histogram Computation of 1024 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 4 blocks of 256 threads
Time on device: 0.033984 ms
Time on CPU: 0.007000 ms
Speed-up: 0.21
Copy output data from CUDA device to host memory
GPU / CPU Histogram First 10 Computations Comparison
Bins	GPU	CPU
Bin[1]: 0 	 0
Bin[2]: 12 	 12
Bin[3]: 8 	 8
Bin[4]: 12 	 12
Bin[5]: 8 	 8
Bin[6]: 9 	 9
Bin[7]: 15 	 15
Bin[8]: 11 	 11
Bin[9]: 12 	 12
Bin[10]: 10 	 10

Done


## Executing the code with an input value of N = 2^12


In [41]:
!./computeHistogram 2 12

Histogram Computation of 4096 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 16 blocks of 256 threads
Time on device: 0.040928 ms
Time on CPU: 0.018000 ms
Speed-up: 0.44
Copy output data from CUDA device to host memory
GPU / CPU Histogram First 10 Computations Comparison
Bins	GPU	CPU
Bin[1]: 0 	 0
Bin[2]: 35 	 35
Bin[3]: 34 	 34
Bin[4]: 36 	 36
Bin[5]: 52 	 52
Bin[6]: 33 	 33
Bin[7]: 42 	 42
Bin[8]: 45 	 45
Bin[9]: 39 	 39
Bin[10]: 46 	 46

Done


## Executing the code with an input value of N = 2^14

In [42]:
!./computeHistogram 2 14

Histogram Computation of 16384 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 64 blocks of 256 threads
Time on device: 0.034816 ms
Time on CPU: 0.054000 ms
Speed-up: 1.55
Copy output data from CUDA device to host memory
GPU / CPU Histogram First 10 Computations Comparison
Bins	GPU	CPU
Bin[1]: 0 	 0
Bin[2]: 149 	 149
Bin[3]: 187 	 187
Bin[4]: 162 	 162
Bin[5]: 194 	 194
Bin[6]: 173 	 173
Bin[7]: 158 	 158
Bin[8]: 170 	 170
Bin[9]: 155 	 155
Bin[10]: 151 	 151

Done


## Conclusion
In conclusion, the CUDA program successfully leverages GPU parallelism with shared memory to compute a histogram, demonstrating accelerated processing compared to the CPU. The comparison of elapsed times indicates the performance advantage of GPU computing, especially for larger workloads. The first ten histogram values, printed for both CPU and GPU, showcase the consistency of results. The code's flexibility in handling different input sizes highlights its adaptability to varying computational demands. Considerations for optimization and future enhancements underscore the ongoing potential for refining performance based on specific application needs and hardware capabilities.